In [11]:
from os import walk, system
import json
import pandas as pd

In [12]:
schools = json.load(open('out/2022/school_url_pages.json'))

In [3]:
from helpers.school import SchoolData

In [4]:
fpath = school_pages['computer_internet']
tables = pd.read_html(fpath)
for tab in tables:
    bool_tab = ('จำนวนคอมพิวเตอร์เพื่อการเรียนการสอน' == tab)
    bool_tab.any().any()
    if bool_tab.any().any():
        df = tab
        break
df

NameError: name 'school_pages' is not defined

In [ ]:

tables = pd.read_html(school_pages['computer_internet'])
for table in tables:
  if any('จำนวนคอมพิวเตอร์เพื่อการเรียนการสอน' == table):
    df = table

In [ ]:
df

,0,1
0,ระบบเครือข่ายอินเทอร์เน็ตที่โรงเรียนเช่าเอง,ระบบเครือข่ายอินเทอร์เน็ตที่โรงเรียนเช่าเอง
1,ผู้ให้บริการ:,-
2,ประเภท:,-
3,ความเร็ว:,-
4,สถานะการใช้งาน:,-
5,งบประมาณ/เดือน:,-
6,ระบบเครือข่าย,ระบบเครือข่าย
7,ระบบเครือข่าย (LAN):,มีระบบเครือข่าย LAN
8,ระบบเครือข่ายไร้สาย (Wireless LAN):,มีระบบเครือข่ายไร้สาย (Wireless LAN)
9,ข้อมูล ณ วันที่ 2021-09-03 18:39:38 น.,ข้อมูล ณ วันที่ 2021-09-03 18:39:38 น.


In [ ]:
pd.read_csv('out/2022-09/school_data/internet.csv')

,Unnamed: 0,Unnamed: 0.1,0,1,2,3,school_id,0.1,1.1,2.1,3.1
0,0,0.0,NaN,ยินดีต้อนรับ,NaN,หน้าหลักเขตพื้นที่ | admin page,1010720019,NaN,NaN,NaN,NaN
1,1,NaN,NaN,ยินดีต้อนรับ,NaN,หน้าหลักเขตพื้นที่ | admin page,1010720007,NaN,NaN,NaN,NaN
2,2,NaN,NaN,ยินดีต้อนรับ,NaN,หน้าหลักเขตพื้นที่ | admin page,1010720023,NaN,NaN,NaN,NaN
3,3,NaN,NaN,ยินดีต้อนรับ,NaN,หน้าหลักเขตพื้นที่ | admin page,1010720003,NaN,NaN,NaN,NaN
4,4,NaN,NaN,ยินดีต้อนรับ,NaN,หน้าหลักเขตพื้นที่ | admin page,1010720004,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
29354,29354,NaN,NaN,NaN,NaN,NaN,1018090200,NaN,ยินดีต้อนรับ,NaN,หน้าหลักเขตพื้นที่ | admin page
29355,29355,NaN,NaN,NaN,NaN,NaN,1018090198,NaN,ยินดีต้อนรับ,NaN,หน้าหลักเขตพื้นที่ | admin page
29356,29356,NaN,NaN,NaN,NaN,NaN,1018090201,NaN,ยินดีต้อนรับ,NaN,หน้าหลักเขตพื้นที่ | admin page
29357,29357,NaN,NaN,NaN,NaN,NaN,1018090205,NaN,ยินดีต้อนรับ,NaN,หน้าหลักเขตพื้นที่ | admin page


# Computer and internet

In [ ]:
import pandas as pd

In [ ]:
dfs = pd.read_html('out/2022/html/school/computer_internet/1010720040.html')

def _df_to_dict(df):
    header = ''
    rows = dict()
    df
    for i, row in df.iterrows():
        if len(row.unique()) != len(row):
            header = row[0]
        else:
            row_list = row.values.tolist()
            col, cell = row_list
            if col not in rows.keys():
                rows[col] = dict()
            rows[col][header] = cell
    return rows
rows = _df_to_dict(dfs[5])
pd.DataFrame(rows)

,index,งบประมาณ สพฐ.,จัดหาเอง/บริจาค:,รวม,ใช้งานได้:,ใช้งานไม่ได้:,คอมพิวเตอร์:
0,จำนวนคอมพิวเตอร์เพื่อการเรียนการสอน,245 เครื่อง,111 เครื่อง,356 เครื่อง,356 เครื่อง,0 เครื่อง,NaN
1,จำนวนคอมพิวเตอร์เพื่อการบริหารจัดการ,0 เครื่อง,0 เครื่อง,0 เครื่อง,0 เครื่อง,0 เครื่อง,NaN
2,จำนวนคอมพิวเตอรทั้งหมด,NaN,NaN,NaN,356 เครื่อง,0 เครื่อง,356 เครื่อง


In [ ]:
def _internet_helper(df):
    header = ''
    rows = dict()
    for i, row in df.iterrows():
        if len(row.unique()) != len(row):
            header = row[0]
        else:
            row_list = row.values.tolist()
            col, cell = row_list
            rows[header+'_'+col] = cell
    df = pd.DataFrame([rows.values()], columns=rows.keys())
    return df


,ระบบเครือข่ายอินเทอร์เน็ตที่โรงเรียนเช่าเอง.ผู้ให้บริการ:,ระบบเครือข่ายอินเทอร์เน็ตที่โรงเรียนเช่าเอง.ประเภท:,ระบบเครือข่ายอินเทอร์เน็ตที่โรงเรียนเช่าเอง.ความเร็ว:,ระบบเครือข่ายอินเทอร์เน็ตที่โรงเรียนเช่าเอง.สถานะการใช้งาน:,ระบบเครือข่ายอินเทอร์เน็ตที่โรงเรียนเช่าเอง.งบประมาณ/เดือน:,ระบบเครือข่าย.ระบบเครือข่าย (LAN):,ระบบเครือข่าย.ระบบเครือข่ายไร้สาย (Wireless LAN):
0,3BB,fiber optic,100/100 Mbps,OK Mbps,5000,มีระบบเครือข่าย LAN,มีระบบเครือข่ายไร้สาย (Wireless LAN)


# Durable Goods

In [ ]:
from src.indexer import Index

In [ ]:
index = Index('out/2022/result_index.txt')

In [ ]:
def read_first_line(fpath):
  with open(fpath, 'r') as file:
    line = ''
    c = ''
    while c != '\n':
      c = file.read(1)
      line += c
    return line

In [ ]:
%time
read_first_line('out/2022/url_index.txt')

CPU times: user 3 µs, sys: 1e+03 ns, total: 4 µs
Wall time: 16 µs


'https://bobec.bopp-obec.info/build_sch_list.php?Area_CODE=100001\tout/2022/html/area/100001.html\n'

In [ ]:
durable_goods = [x for x in index.index.keys() if x.split('.')[1] == 'durable_goods']
headers = {x: read_first_line(index[x]) for x in durable_goods}

In [ ]:
def uniq(items):
  unique_items = []
  for i in items:
    if i not in unique_items:
      unique_items.append(i)
  return unique_items

uniq(headers.values())

['ลำดับ,รหัส,รายการ,รายการ(cm.),จำนวนที่ใช้ได้,จำนวนรอซ่อม,จำนวนรอจำหน่าย\n']

In [ ]:
target = '<!-- .style37 {color: #FFFFFF; font-family: Tahoma; } .style40 {font-family: Tahoma; font-size: 12px; } -->  เมนูหลัก  หน้าหลักเขตพื้นที่  ข้อมูลพื้นฐานโรงเรียน  ข้อมูลนักเรียน  ข้อมูลครูบุคลากร  ข้อมูลคอมพิวเตอร์ อินเทอร์เน็ต  ข้อมูลสิ่งก่อสร้าง  ข้อมูลครุภัณฑ์  เขตพื้นที่บริการ  ข่าวประชาสัมพันธ์,ข้อมูลครุภัณฑ์ (M-OBEC) โรงเรียนราชสีมาวิทยาลัย\n'

for k, x in headers.items():
  if x == target:
    break

In [ ]:
index[k]

'out/2022/school_data/school/1030201355/durable_goods.csv'

# Rename header

In [ ]:
filepaths = [
  'out/2022/school_data/building.csv',
  'out/2022/school_data/computer.csv',
  'out/2022/school_data/durable_goods.csv',
  'out/2022/school_data/general.csv',
  'out/2022/school_data/internet.csv',
  'out/2022/school_data/staff.csv',
  'out/2022/school_data/student.csv'
]

In [ ]:
# headers = [read_first_line(fp) for fp in filepaths]

In [ ]:

with open('header_map.json') as fp:
  mapper = json.load(fp)

In [ ]:
dir = 'building'
df = pd.read_csv(f'out/2022/school_data/{dir}.csv', index_col=0)

In [ ]:
dir_mapper = mapper[dir]
dir_mapper = {old: new for old, new in dir_mapper.items() if new}
df[['school_id'] + [col for col in df.columns if col != 'school_id']]\
  .rename(dir_mapper, axis=1)\
  .fillna('-')

,school_id,name,model,room_number,build_at,current_condition,image_url_0,image_url_1
0,1010720019,อาคารเรียน ลำดับที่ 1,004,18 ห้อง,พ.ศ. 2497,ดี,https://bobec.bopp-obec.info/b01/7200191012124...,https://bobec.bopp-obec.info/b01/7200191012124...
1,1010720019,อาคารเรียน ลำดับที่ 2,"108 ล,108 ,ล/30,108 หรือ108 ชื่ออื่นๆ",15 ห้อง,พ.ศ. 2528,ดี,https://bobec.bopp-obec.info/b01/7200192012125...,https://bobec.bopp-obec.info/b01/7200192012125...
2,1010720019,บ้านพักภารโรง ลำดับที่ 1,อื่น อื่น,-,พ.ศ. 2538,ดี,https://bobec.bopp-obec.info/b04/7200191040225...,https://bobec.bopp-obec.info/b04/7200191040225...
3,1010720007,อาคารเรียน ลำดับที่ 1,"อื่นๆ 004,008,030,107,206,210,214,315,415,308",24 ห้อง,พ.ศ. 2516,พอใช้,https://bobec.bopp-obec.info/b01/7200071012125...,https://bobec.bopp-obec.info/b01/7200071012125...
4,1010720007,อาคารเรียน ลำดับที่ 2,"อื่นๆ 004,008,030,107,206,210,214,315,415,308",24 ห้อง,พ.ศ. 2529,พอใช้,https://bobec.bopp-obec.info/b01/7200072012125...,https://bobec.bopp-obec.info/b01/7200072012125...
...,...,...,...,...,...,...,...,...
402708,1018090206,อาคารอเนกประสงค์/โรงอาหาร/โรงฝึกงาน/หอประชุม ล...,สปช.202/26,-,พ.ศ. 2530,ดี,https://bobec.bopp-obec.info/b02/9020610202253...,https://bobec.bopp-obec.info/b02/9020610202253...
402709,1018090206,บ้านพักครู ลำดับที่ 1,กรมสามัญ,-,พ.ศ. 2516,พอใช้,https://bobec.bopp-obec.info/b03/9020610304251...,https://bobec.bopp-obec.info/b03/9020610304251...
402710,1018090206,บ้านพักครู ลำดับที่ 2,บ้านพักครู 203/27,-,พ.ศ. 2561,ดี,https://bobec.bopp-obec.info/b03/1801020520309...,https://bobec.bopp-obec.info/b03/1801020520309...
402711,1018090206,ส้วม ลำดับที่ 1,อาคาร สพฐ. 4(ส้วม 4 ที่นั่ง),-,พ.ศ. 2560,พอใช้,https://bobec.bopp-obec.info/b05/1801020560514...,https://bobec.bopp-obec.info/b05/1801020560514...


In [ ]:
map[dir]

{'school_id': '',
 'name': '',
 'แบบ': 'model',
 'จำนวนห้องตามแปลน': 'room_number',
 'สร้าง': 'build_at',
 'สภาพปัจจุบัน': 'current_condition',
 'image_url_0': '',
 'image_url_1': ''}

In [5]:
# JSON Format

In [1]:
import helpers.school as sh
from helpers.utils import *
import re

In [13]:
school_ids = list(schools.keys())

In [14]:
schools_pages_fpath = os.path.join(ROOT_DIR, 'school_file_path_pages.json')
schools_pages = load_json(schools_pages_fpath)
building_index_fpath = os.path.join(ROOT_DIR, 'building_index.txt')
buildings = Index(building_index_fpath)

In [15]:
from typing import Union

In [171]:
mappers = load_json('header_mapper.json')

In [172]:
def rename(to_rename, data_dir, axis=1):
    mappers = load_json('header_mapper.json')
    mapper = mappers[data_dir]
    if isinstance(to_rename, pd.DataFrame):
        return to_rename.rename(mapper, axis=axis)
    raise TypeError()

def replace(to_rename, data_dir):
    mappers = load_json('header_mapper.json')
    mapper = mappers[data_dir]
    if isinstance(to_rename, pd.DataFrame):
        return to_rename.replace(mapper)
    if isinstance(to_rename, str):
        return mapper[to_rename]
    raise TypeError()

In [285]:
class SchoolData:
    def __init__(self, school_id: str, page_fpaths: Dict) -> None:
        self.school_id = school_id
        self.pages: Dict = page_fpaths
        self.save_dir = os.path.join(
            ROOT_DIR, 'school_data', 'school', school_id)

    def general(self) -> Dict:
        about_school: Dict = dict()
        about_school_json: List = dict()
        soup: BeautifulSoup = load_soup(self.pages['general'])
        table = soup.find('table').find('table', attrs={'width': '521'})
        for comment in soup.children:
            break
        url: str = re.findall('url: (.*)\n', comment)[0]

        parent_url: str = re.sub('[^/]*$', '', url)

        for table_row in table.find_all('tr'):
            cells = table_row.find_all('td')

            if len(cells) == 2:
                key_name: str = clean_text(cells[0].text)
                value: str = clean_text(cells[1].text)

                if cells[1].find_all('a'):
                    links = []
                    for a_tag in cells[1].find_all('a'):
                        if not a_tag.text:
                            continue
                        links.append({
                            "text": a_tag.text,
                            "url": (parent_url
                                    if not a_tag.attrs['href'].startswith('http')
                                    else '') + a_tag.attrs['href']
                        })
                    about_school_json.update({"links": links})
                    continue

                about_school_json[replace(key_name, 'general')] = value

        image_dir: str = SCRAPED_FILE_DIRS['general']+'/image'

        if not is_path_existed(image_dir):
            makedirs(image_dir)

        for div in soup.find_all('div'):
            if not div.find('div'):
                if 'ผู้อำนวยการโรงเรียน' in div.text:
                    image = div.find('img')
                    about_school_json['principal'] = {
                        'name': div.text.strip().split('\n')[0],
                        'position_title': div.text.strip().split('\n')[1].strip()
                    }
                    image = div.find('img')
                    if image is not None:
                        image_src = image.attrs['src']
                        about_school_json['principal']['image_path'] = parent_url + image_src

                elif 'ตราสัญลักษณ์' in div.text:
                    image = div.find('img')
                    if image is not None:
                        image_src = image.attrs['src']
                        about_school_json['logo_image_path'] = parent_url + image_src

        about_school_json['latlng'] = None
        js_text: str = ' '.join(
            [script.text for script in soup.find_all('script')])
        latlng: str = re.findall('LatLng\((.*)\)', js_text)
        if latlng and ',' in latlng[0]:
            about_school_json['latlng'] = latlng[0].split(',')

        return about_school_json


    def computer(self) -> Dict:
        df = self._find_html_table(
            'computer_internet', 'จำนวนคอมพิวเตอร์เพื่อการเรียนการสอน')
        if df is None:
            return dict()
        header = ''
        rows = dict()
        df = replace(df, 'computer')
        for _, row in df.iterrows():
            if len(row.unique()) != len(row):
                header = row[0]
            else:
                if header not in rows.keys():
                    rows[header] = dict()
                row_list = row.values.tolist()
                col, cell = row_list
                cell = re.sub(r'(\d+) ?เครื่อง', r'\1', cell)
                cell = int(cell)
                if col.startswith('source'):
                    source_dict = rows[header].get('source', dict())
                    source_dict[col.replace('source_','')] = cell
                    rows[header]['source'] = source_dict
                else:
                    rows[header][col] = cell
        dict_total = rows['total']
        del rows['total']
        rows.update(dict_total)
        return rows

    def _find_html_table(self, page, keyword):
        fpath = self.pages[page]
        tables = pd.read_html(fpath)
        for table in tables:
            table_bool = (keyword == table)
            if table_bool.any().any():
                df = table
                return df


    def save(self) -> Dict[str, Dict[str, Union[Dict, pd.DataFrame]]]:
        makedirs(self.save_dir, exist_ok=True)

        methods = {
            'building': self.building,
            'computer': self.computer,
            'durable_goods': self.durable_goods,
            'general': self.general,
            'student': self.student,
            'staff': self.staff,
            'internet': self.internet,
        }
        temp: Dict = dict()

        for dir in self.pages:
            if dir == 'computer_internet':
                for dir in ['computer', 'internet']:
                    data = self.wirte_file(methods[dir], dir)
                    temp[dir] = data
            else:
                data = self.wirte_file(methods[dir], dir)
                temp[dir] = data
        return temp

    def wirte_file(self, caller, name):
        fpath = os.path.join(self.save_dir, f'{name}.json')
        ix = self.school_id + '.' + name

        data: Dict = caller()
        if data is not None :
            dump_json(data, fpath)
            data_index[ix] = fpath
            return data


In [286]:
school_index = '101431089'
school_data = SchoolData(school_index,
                         {**schools_pages[school_index], 'building':buildings[school_index[4:]]})

In [287]:
df = school_data.computer()
df

{'learning': {'source': {'obec': 7, 'self': 38},
  'total': 45,
  'working': 30,
  'broken': 10},
 'management': {'source': {'obec': 4, 'self': 0},
  'total': 4,
  'working': 4,
  'broken': 0},
 'total': 49,
 'working': 34,
 'broken': 10}

# สังกัด

In [6]:
from helpers.province import PROVINCE_HTML_DIR
import os
import pandas as pd

In [32]:
all_dfs = []
for province_html in os.listdir(PROVINCE_HTML_DIR):
    if 'index.html' == province_html: continue
    file_path = os.path.join(PROVINCE_HTML_DIR, province_html)
    
    tables = pd.read_html(file_path, converters={'รหัสโรงเรียน': lambda x: int(x), 'รหัส สพท.': lambda x: int(x)})
    
    indexer = tables[0].dropna()
    
    # indexer.loc[:, 'รหัสโรงเรียน'] = indexer['รหัสโรงเรียน'].astype(int)
    # indexer.loc[:, 'รหัส สพท.'] = indexer['รหัส สพท.'].astype(int)
    indexer.set_index('รหัสโรงเรียน', inplace=True)
    all_dfs.append(indexer)

In [33]:
affiliation_df = pd.concat(all_dfs)

In [44]:
affiliation_df['สพท.']

รหัสโรงเรียน
94013001          สำนักบริหารงานการศึกษาพิเศษ
94012001    สพม.เขต 15(นราธิวาส-ปัตตานี-ยะลา)
94012002    สพม.เขต 15(นราธิวาส-ปัตตานี-ยะลา)
94012003    สพม.เขต 15(นราธิวาส-ปัตตานี-ยะลา)
94012004    สพม.เขต 15(นราธิวาส-ปัตตานี-ยะลา)
                          ...                
76020130                   สพป.เพชรบุรี เขต 2
76020131                   สพป.เพชรบุรี เขต 2
76020132                   สพป.เพชรบุรี เขต 2
76020133                   สพป.เพชรบุรี เขต 2
76020134                   สพป.เพชรบุรี เขต 2
Name: สพท., Length: 29576, dtype: object

In [42]:
affiliation_df

,ลำดับ,รหัส สพท.,สพท.,โรงเรียน,ประเภท,ผู้บริหาร
รหัสโรงเรียน,,,,,,
94013001,1,100700,สำนักบริหารงานการศึกษาพิเศษ,ราชประชานุเคราะห์ 40 จังหวัดปัตตานี,ประถมศึกษา-มัธยมศึกษาตอนปลาย,นายชาญชัย สุดใจ
94012001,2,101715,สพม.เขต 15(นราธิวาส-ปัตตานี-ยะลา),เบญจมราชูทิศ จังหวัดปัตตานี,มัธยมศึกษาตอนต้น-มัธยมศึกษาตอนปลาย,นายประสพ ชนามุยา
94012002,3,101715,สพม.เขต 15(นราธิวาส-ปัตตานี-ยะลา),เดชะปัตตนยานุกูล,มัธยมศึกษาตอนต้น-มัธยมศึกษาตอนปลาย,นายสมพงค์ สัจจาภรณ์
94012003,4,101715,สพม.เขต 15(นราธิวาส-ปัตตานี-ยะลา),ท่าข้ามวิทยาคาร,มัธยมศึกษาตอนต้น-มัธยมศึกษาตอนปลาย,นายสมชาย ขวัญมนิจ
94012004,5,101715,สพม.เขต 15(นราธิวาส-ปัตตานี-ยะลา),วุฒิชัยวิทยา,มัธยมศึกษาตอนต้น-มัธยมศึกษาตอนปลาย,นายจำนงค์ แสงหวัง
...,...,...,...,...,...,...
76020130,234,7602,สพป.เพชรบุรี เขต 2,บ้านสองพี่น้อง (สายปัญญาสมาคมในพระบรมราชินูปถั...,อนุบาล-ประถมศึกษา,นางสาวณหฤทัย จิรธนชยกุล
76020131,235,7602,สพป.เพชรบุรี เขต 2,บ้านหนองปืนแตก,อนุบาล-มัธยมศึกษาตอนต้น,ว่าที่ร้อยโทณัฐพัชร เถกิงกร
76020132,236,7602,สพป.เพชรบุรี เขต 2,บ้านด่านโง,อนุบาล-มัธยมศึกษาตอนต้น,นายอนุรักษ์ ทับสมบัติ


In [40]:
len('94013001')

8

In [1]:
from helpers.utils import SchoolDataIndex

In [2]:
sdi = SchoolDataIndex()

In [4]:
sdi.get_school('1020080309')

{'ลำดับ': '2',
 'รหัส สพท.': '101718',
 'สพท.': 'สพม.เขต 18(ชลบุรี-ระยอง)',
 'รหัสโรงเรียน': '20012002',
 'โรงเรียน': 'ชลราษฎรอำรุง',
 'href': 'schooldata-view.php?School_ID=1020080309&Area_CODE=101718',
 'ประเภท': 'มัธยมศึกษาตอนต้น-มัธยมศึกษาตอนปลาย',
 'ผู้บริหาร': 'นางสาวรุ่งทิพย์   พรหมศิริ'}